In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../src')

In [3]:
import torch
import scipy.sparse as sp
import glob
import warnings
import anndata
import matplotlib.pyplot as plt
import scanpy as sc
import numpy as np
import seaborn as sns
import torch
import pyro

In [4]:
import spaceoracle
from spaceoracle import SpaceOracle

In [5]:
adata_train = sc.read_h5ad('./.cache/adata_train.h5ad')
adata_train

AnnData object with n_obs × n_vars = 10277 × 4998
    obs: 'cluster', 'rctd_cluster', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'n_counts', 'spatial_kmeans'
    var: 'mt', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'cluster_colors', 'commot-user_database-info', 'flowsig_network', 'hvg', 'log1p', 'nsf_info', 'rctd_cluster_colors'
    obsm: 'X_flow', 'X_gem', 'X_spatial', 'commot-user_database-sum-receiver', 'commot-user_database-sum-sender', 'rctd_results', 'spatial'
    layers: 'imputed_count', 'normalized_count', 'raw_count'
    obsp: 'commot-user_database-Angptl2-Pirb', 'commot-user_database-Angptl4-Cdh5

In [3]:
import shutil
import os
    
models_dir = './models'
if os.path.exists(models_dir):
    shutil.rmtree(models_dir)
os.makedirs(models_dir)

In [10]:
sc.pp.highly_variable_genes(adata_train, n_top_genes=100, subset=True, inplace=True)
adata_train

AnnData object with n_obs × n_vars = 10277 × 100
    obs: 'cluster', 'rctd_cluster', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'n_counts', 'spatial_kmeans'
    var: 'mt', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'cluster_colors', 'commot-user_database-info', 'flowsig_network', 'hvg', 'log1p', 'nsf_info', 'rctd_cluster_colors'
    obsm: 'X_flow', 'X_gem', 'X_spatial', 'commot-user_database-sum-receiver', 'commot-user_database-sum-sender', 'rctd_results', 'spatial'
    layers: 'imputed_count', 'normalized_count', 'raw_count'
    obsp: 'commot-user_database-Angptl2-Pirb', 'commot-user_database-Angptl4-Cdh5'

In [11]:
so = spaceoracle.SpaceOracle(
    adata=adata_train,
    annot='rctd_cluster', 
    max_epochs=7, 
    learning_rate=3e-4, 
    spatial_dim=64,
    batch_size=512,
    alpha=0.9,
)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


🌍️ Generating spatial 64x64 maps...


In [12]:
so.run()

Fitting 7 models in parallel... with 7/11 cores


/opt/homebrew/Caskroom/miniforge/base/envs/SpaceOracle/lib/python3.9/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.B

2 regulators + 0 ligand-receptor pairs
Fitting 7 models in parallel... with 7/11 cores
2 regulators + 0 ligand-receptor pairs
Fitting 7 models in parallel... with 7/11 cores
2 regulators + 0 ligand-receptor pairs
Fitting 7 models in parallel... with 7/11 cores
1 regulators + 0 ligand-receptor pairs
Fitting 7 models in parallel... with 7/11 cores
2 regulators + 0 ligand-receptor pairs
Fitting 7 models in parallel... with 7/11 cores
2 regulators + 0 ligand-receptor pairs
Fitting 7 models in parallel... with 7/11 cores
1 regulators + 0 ligand-receptor pairs
Fitting 7 models in parallel... with 7/11 cores
2 regulators + 0 ligand-receptor pairs
Fitting 7 models in parallel... with 7/11 cores
2 regulators + 0 ligand-receptor pairs
Fitting 7 models in parallel... with 7/11 cores
1 regulators + 0 ligand-receptor pairs
Fitting 7 models in parallel... with 7/11 cores
3 regulators + 0 ligand-receptor pairs
Fitting 7 models in parallel... with 7/11 cores
2 regulators + 0 ligand-receptor pairs
Fitt